# Описание проекта: Промышленность

Чтобы оптимизировать производственные расходы, металлургический комбинат «Стальная птица» решил уменьшить потребление электроэнергии на этапе обработки стали.\
Для этого комбинату нужно контролировать температуру сплава. \
Ваша задача — построить модель, которая будет её предсказывать. \
Заказчик хочет использовать разработанную модель для имитации технологического процесса. \
Изучите его, прежде чем генерировать новые признаки.


## Описание процесса обработки

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн.\
Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом.\
Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами.\
Они установлены на крышке ковша.

Сначала происходит десульфурация — из стали выводят серу и корректируют её химический состав добавлением примесей.\
Затем сталь легируют — добавляют в неё куски сплава из бункера для сыпучих материалов или порошковую проволоку через специальный трайб-аппарат.

Прежде чем в первый раз ввести легирующие добавки, специалисты производят химический анализ стали и измеряют её температуру.\
Потом температуру на несколько минут повышают, уже после этого добавляют легирующие материалы и продувают сталь инертным газом, чтобы перемешать, а затем снова проводят измерения. \
Такой цикл повторяется до тех пор, пока не будут достигнуты нужный химический состав стали и оптимальная температура плавки.


Дальше расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. \
Оттуда готовый продукт выходит в виде заготовок-слябов (англ. *slab*, «плита»).

## Описание данных


Данные хранятся в `Sqlite` — СУБД, в которой база данных представлена одним файлом. Она состоит из нескольких таблиц:

- `steel.data_arc` — данные об электродах;
- `steel.data_bulk` — данные об объёме сыпучих материалов;
- `steel.data_bulk_time` — данные о времени подачи сыпучих материалов;
- `steel.data_gas` — данные о продувке сплава газом;
- `steel.data_temp` — данные об измерениях температуры;
- `steel.data_wire` — данные об объёме проволочных материалов;
- `steel.data_wire_time` — данные о времени подачи проволочных материалов.

### Таблица `steel.data_arc`

- `key` — номер партии;
- `BeginHeat` — время начала нагрева;
- `EndHeat` — время окончания нагрева;
- `ActivePower` — значение активной мощности;
- `ReactivePower` — значение реактивной мощности.

### Таблица `steel.data_bulk`

- `key` — номер партии;
- `Bulk1` … `Bulk15` — объём подаваемого материала.

### Таблица `steel.data_bulk_time`

- `key` — номер партии;
- `Bulk1` … `Bulk15` — время подачи материала.

#### Таблица `steel.data_gas`

- `key` — номер партии;
- `gas` — объём подаваемого газа.

#### Таблица `steel.data_temp`

- `key` — номер партии;
- `MesaureTime` — время замера;
- `Temperature` — значение температуры.

#### Таблица `steel.data_wire`

- `key` — номер партии;
- `Wire1` … `Wire9` — объём подаваемых проволочных материалов.

#### Таблица `steel.data_wire_time`

- `key` — номер партии;
- `Wire1` … `Wire9` — время подачи проволочных материалов.

Во всех файлах столбец `key` содержит номер партии. В таблицах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.


## План проекта: Прогнозирование температуры стали

### **Шаг 1. Загрузка данных**

1. Подключение к базе данных в SQLite.
2. Загрузка данных из базы данных.
3. Установка константы RANDOM_STATE. Значение равно дате начала проекта (например, RANDOM_STATE = 10922 для 1 сентября 2022 года).

### **Шаг 2. Исследовательский анализ и предобработка данных**

Общие задачи:
1. Проверка всех таблиц на наличие данных и соответствие количеству таблиц условию задачи.

**Анализ таблицы steel.data_arc:**

1. Проведение исследовательского анализа данных (EDA):
    - Проверка наличия пропусков и аномалий.
    - Изучение распределения признаков.
2. Обработка аномалий:
    - Удаление партий с аномальными значениями или замена аномальных значений.
3. Генерация новых признаков:
    - Длительность нагрева.
    - Общая мощность.
    - Соотношение активной мощности к реактивной.
    - Количество запусков нагрева электродами.
4. Агрегация наблюдений по ключу после генерации новых признаков.

**Анализ таблицы steel.data_bulk:**
1. Проведение EDA:
    - Проверка наличия пропусков и аномалий.
    - Изучение распределения признаков.
2. Обработка пропусков:
    - Пропуск означает, что материал не добавляли в партию.

**Анализ таблицы steel.data_bulk_time:**
1. Проведение EDA:
    - Проверка наличия пропусков и аномалий.
    - Изучение распределения признаков.
2. Проверка данных на адекватность (например, что подача материала не измеряется сутками).

**Анализ таблицы steel.data_gas:**
1. Проведение EDA:
    - Проверка наличия пропусков и аномалий.
    - Изучение распределения признаков.

**Анализ таблицы steel.data_temp:**
1. Проведение EDA:
    - Проверка наличия пропусков и аномалий.
    - Изучение распределения признаков.
2. Целевой признак:
    - Последняя температура партии.
3. Использование начальной температуры партии как входного признака.
4. Исключение промежуточных значений температуры для предотвращения утечки целевого признака.
5. Агрегация наблюдений:
    - Учитываются только ключи с как минимум двумя наблюдениями: первый и последний замеры температуры.
6. Обработка аномальных значений:
    - Температуры ниже 1500 градусов считаются аномальными.

**Анализ таблицы steel.data_wire:**
1. Обработка пропусков:
    - Пропуск означает, что материал не добавляли в партию.

**Анализ таблицы steel.data_wire_time:**
1. Проведение EDA:
    - Проверка наличия пропусков и аномалий.
    - Изучение распределения признаков.
2. Проверка данных на адекватность (например, что подача материала не измеряется сутками).

**Объединение таблиц по ключу:**
1. Каждой партии должно соответствовать одно наблюдение.
2. Учет всех материалов (сыпучие, проволочные) и процессов (продувка газом, нагревание).
3. Обработка возможной асинхронности времени между датчиками.

**Общий анализ объединенной таблицы:**
1. Проведение EDA объединенной таблицы.
2. Визуализация распределения каждого признака.
3. Корреляционный анализ.

**Подготовка данных для обучения:**
1. Выбор признаков для обучения.
2. Разделение данных на тренировочную и тестовую выборки (`test_size = 0.25`).
3. Подготовка данных для обучения с учетом особенностей выбранных моделей.

### **Шаг 3. Обучение модели**

1. **Рассмотрение классов моделей:**
    - Решающее дерево или случайный лес.
    - Бустинги.
    - Нейронные сети.
2. Обучение моделей и оценка их качества с использованием метрики MAE.
3. Выбор лучшей модели на основе значений метрики на кросс-валидации.
4. Подбор значений гиперпараметров:
    - Использование методов автоматизированного подбора гиперпараметров (`GridSearchCV`, `RandomizedSearchCV`, `OptunaSearchCV`, `Optuna` и другие)



### Шаг 4. Тестирование модели и демонстрация работы

1. Проверка качества лучшей модели на тестовой выборке.
2. Значение метрики MAE должно быть менее 6.8.
3. Дополнительная оценка модели с использованием R².
4. Сравнение результатов лучшей модели и константной модели.
5. Анализ важности основных признаков.
6. Проведение дополнительного исследования для одного из важных признаков:
    - Построение графика зависимости входного и целевого признаков.


### Шаг 5. Общий вывод

1. Написание общих выводов по проекту.
2. Предложение способов для дальнейшего улучшения модели.
3. Формулирование бизнес-рекомендаций заказчику.

## Подготовка данных

### Загрузка библиотек

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, text, select
from sqlalchemy.orm import DeclarativeBase, Session
from sqlalchemy.schema import DropTable
from sqlalchemy.ext.compiler import compiles
from sqlalchemy import MetaData, Table


### Инциализация констант, настройка окружения

In [2]:
# При с работе с проектом необходимо отредактировать константу пути
PATH_TO_DATABASE = 'datasets/ds-plus-final.db'
RANDOM_STATE = 150824
# Создаем объект подключения (engine) к базе данных SQLite
engine = create_engine(f'sqlite:///{PATH_TO_DATABASE}', echo=False)

### Проверка БД

По условию проекта в базе данных должно быть 7 таблиц:\
'data_arc', 'data_bulk', 'data_bulk_time', 'data_gas', 'data_temp', 'data_wire', 'data_wire_time'

Проверим:

In [13]:
# Создаем объект MetaData
metadata = MetaData()

# Отображаем таблицы из базы данных
metadata.reflect(bind=engine)

# Получаем список таблиц
tables = metadata.tables.keys()
print(list(tables))

['data_arc', 'data_bulk', 'data_bulk_time', 'data_gas', 'data_temp', 'data_wire', 'data_wire_time']


In [12]:
tables

dict_keys(['data_arc', 'data_bulk', 'data_bulk_time', 'data_gas', 'data_temp', 'data_wire', 'data_wire_time'])

In [4]:
# Выводим информацию о типах данных в каждой таблице
for table_name in metadata.tables:
    print(f"\nТаблица: {table_name}")
    table = metadata.tables[table_name]
    for column in table.c:
        print(f" - Столбец: {column.name}, Тип данных: {column.type}")


Таблица: data_arc
 - Столбец: key, Тип данных: INTEGER
 - Столбец: Начало нагрева дугой, Тип данных: TEXT
 - Столбец: Конец нагрева дугой, Тип данных: TEXT
 - Столбец: Активная мощность, Тип данных: DOUBLE
 - Столбец: Реактивная мощность, Тип данных: DOUBLE

Таблица: data_bulk
 - Столбец: key, Тип данных: INTEGER
 - Столбец: Bulk 1, Тип данных: DOUBLE
 - Столбец: Bulk 2, Тип данных: DOUBLE
 - Столбец: Bulk 3, Тип данных: DOUBLE
 - Столбец: Bulk 4, Тип данных: DOUBLE
 - Столбец: Bulk 5, Тип данных: TEXT
 - Столбец: Bulk 6, Тип данных: TEXT
 - Столбец: Bulk 7, Тип данных: TEXT
 - Столбец: Bulk 8, Тип данных: TEXT
 - Столбец: Bulk 9, Тип данных: TEXT
 - Столбец: Bulk 10, Тип данных: TEXT
 - Столбец: Bulk 11, Тип данных: TEXT
 - Столбец: Bulk 12, Тип данных: TEXT
 - Столбец: Bulk 13, Тип данных: TEXT
 - Столбец: Bulk 14, Тип данных: TEXT
 - Столбец: Bulk 15, Тип данных: TEXT

Таблица: data_bulk_time
 - Столбец: key, Тип данных: INTEGER
 - Столбец: Bulk 1, Тип данных: TEXT
 - Столбец: Bulk

В БД присутсвуют лишние таблицы:\
"internet", "personal", "phone", "contract"

Удалим их и освободим место на диске после их удаления.

In [5]:
# Отражаем таблицы из базы данных (если еще не было сделано)
metadata.reflect(bind=engine)

# Список таблиц для удаления
tables_to_delete = ["internet", "personal", "phone", "contract"]

# Удаляем таблицы
with engine.connect() as connection:
    for table_name in tables_to_delete:
        if table_name in metadata.tables:
            table = Table(table_name, metadata, autoload_with=engine)
            table.drop(bind=engine, checkfirst=True)  # checkfirst=True проверяет наличие таблицы перед удалением
            print(f"Таблица {table_name} удалена.")
    connection.execute(text("VACUUM;"))
    print('Масто на диске очищено с помощью VACUUM')  

Масто на диске очищено с помощью VACUUM


Создадим подключение к БД заново, проверим таблицы.

In [6]:
metadata = MetaData()
metadata.reflect(bind=engine)
tables = metadata.tables.keys()
print(list(tables))
for table_name in metadata.tables:
    print(f"\nТаблица: {table_name}")
    table = metadata.tables[table_name]
    for column in table.c:
        print(f" - Столбец: {column.name}, Тип данных: {column.type}")

['data_arc', 'data_bulk', 'data_bulk_time', 'data_gas', 'data_temp', 'data_wire', 'data_wire_time']

Таблица: data_arc
 - Столбец: key, Тип данных: INTEGER
 - Столбец: Начало нагрева дугой, Тип данных: TEXT
 - Столбец: Конец нагрева дугой, Тип данных: TEXT
 - Столбец: Активная мощность, Тип данных: DOUBLE
 - Столбец: Реактивная мощность, Тип данных: DOUBLE

Таблица: data_bulk
 - Столбец: key, Тип данных: INTEGER
 - Столбец: Bulk 1, Тип данных: DOUBLE
 - Столбец: Bulk 2, Тип данных: DOUBLE
 - Столбец: Bulk 3, Тип данных: DOUBLE
 - Столбец: Bulk 4, Тип данных: DOUBLE
 - Столбец: Bulk 5, Тип данных: TEXT
 - Столбец: Bulk 6, Тип данных: TEXT
 - Столбец: Bulk 7, Тип данных: TEXT
 - Столбец: Bulk 8, Тип данных: TEXT
 - Столбец: Bulk 9, Тип данных: TEXT
 - Столбец: Bulk 10, Тип данных: TEXT
 - Столбец: Bulk 11, Тип данных: TEXT
 - Столбец: Bulk 12, Тип данных: TEXT
 - Столбец: Bulk 13, Тип данных: TEXT
 - Столбец: Bulk 14, Тип данных: TEXT
 - Столбец: Bulk 15, Тип данных: TEXT

Таблица: data_

Посмотрим первые строки каждой таблицы:

In [7]:
# Функция для вывода первых нескольких строк из таблицы
def print_first_rows(table, num_rows=5):
    with engine.connect() as connection:
        stmt = select(table).limit(num_rows)  # Передаем объект table напрямую в select()
        result = connection.execute(stmt)
        rows = result.fetchall()
        print(f"\nТаблица: {table.name}")
        for row in rows:
            print(row)

# Перебор всех таблиц в базе данных
for table_name in metadata.tables:
    table = metadata.tables[table_name]
    print_first_rows(table, num_rows=2)  # Выводим первые две строки из каждой таблицы


Таблица: data_arc
(1, '2019-05-03 11:02:14', '2019-05-03 11:06:02', 0.30513, 0.211253)
(1, '2019-05-03 11:07:28', '2019-05-03 11:10:33', 0.765658, 0.477438)

Таблица: data_bulk
(1, None, None, None, 43.0, None, None, None, None, None, None, None, '206.0', None, '150.0', '154.0')
(2, None, None, None, 73.0, None, None, None, None, None, None, None, '206.0', None, '149.0', '154.0')

Таблица: data_bulk_time
(1, None, None, None, '2019-05-03 11:28:48', None, None, None, None, None, None, None, '2019-05-03 11:24:31', None, '2019-05-03 11:14:50', '2019-05-03 11:10:43')
(2, None, None, None, '2019-05-03 11:36:50', None, None, None, None, None, None, None, '2019-05-03 11:53:30', None, '2019-05-03 11:48:37', '2019-05-03 11:44:39')

Таблица: data_gas
(1, 29.7499859302)
(2, 12.5555609779)

Таблица: data_temp
(1, '2019-05-03 11:02:04', '1571.0')
(1, '2019-05-03 11:07:18', '1604.0')

Таблица: data_wire
(1, 60.059998, None, None, None, None, None, None, None, None)
(2, 96.052315, None, None, None, 

Создадим для каждой таблицы из БД объект pandas. \
Пренесем данный из БД.

In [14]:
# Загрузка таблиц в отдельные DataFrame
for table_name in list(tables):
    with engine.connect() as connection:
        globals()[table_name] = pd.read_sql_table(table_name, connection)
        print(f"Таблица {table_name} загружена в DataFrame.")

Таблица data_arc загружена в DataFrame.
Таблица data_bulk загружена в DataFrame.
Таблица data_bulk_time загружена в DataFrame.
Таблица data_gas загружена в DataFrame.
Таблица data_temp загружена в DataFrame.
Таблица data_wire загружена в DataFrame.
Таблица data_wire_time загружена в DataFrame.


Когда объявляешь переменную через globals()\
в VScode эта переменная подчеркивается как та,\
которая еще не была определена,\
но работать с ней можно.

То есть это не кретично и работе не мешает,\
но меня это немного бесит!

Поэтому воспользуемся таким костылем:

In [16]:
data_arc = data_arc
data_bulk = data_bulk
data_bulk_time = data_bulk_time
data_gas = data_gas
data_temp = data_temp
data_wire = data_wire
data_wire_time = data_wire_time

In [ ]:

# Пример использования загруженных DataFrame
print(data_arc.head())  # Вывод первых 5 строк таблицы data_arc
print(data_bulk.head())  # Вывод первых 5 строк таблицы data_bulk